# Data Preprocessing
This notebook contains codes to be run after retrieving Spotify data from `get_spotify_data.ipynb`.

In [1]:
import json
import pandas as pd

In [2]:
df_audio_analysis = pd.read_csv('./data/raw/audio_analysis.csv')
df_audio_analysis.head()

,track.id,track.name,batch,bars,beats,sections,segments,tatums,meta.analyzer_version,meta.platform,...,track.mode,track.mode_confidence,track.codestring,track.code_version,track.echoprintstring,track.echoprint_version,track.synchstring,track.synch_version,track.rhythmstring,track.rhythm_version
0,4CPYZtb4tX2V03jcsJAZCD,Where's Kevin (From 'Overcooked! 2'),1,"[{'start': 0.68467, 'duration': 2.18383, 'conf...","[{'start': 0.13989, 'duration': 0.54478, 'conf...","[{'start': 0.0, 'duration': 18.13542, 'confide...","[{'start': 0.0, 'duration': 0.13338, 'confiden...","[{'start': 0.13989, 'duration': 0.27239, 'conf...",4.0.0,Linux,...,0,0.253,eJxVmVuC5CgMBK_iI_AUcP-LTYRwV9V87GzjwjZIqVQmXn...,3.15,eJzlnQuOJDmPpK_kekvH0fP-R9jPGD0V1QmEcgu5tY3FDj...,4.12,eJxNlwmSLDkIQ6_iI6TB6_0vNtLD1X-iK7oqnV5ACIEjvv...,1.0,eJyNWgm2HDEKu0odwbvN_S82RgLs6vqZmfeSdKcWLyCEwJ...,1.0
1,4cmRCH5q4Mp5DKqsGkQ2eu,"Super Mario Theme (From ""Super Mario"")",1,"[{'start': 1.36865, 'duration': 2.39713, 'conf...","[{'start': 0.76593, 'duration': 0.60272, 'conf...","[{'start': 0.0, 'duration': 16.95381, 'confide...","[{'start': 0.0, 'duration': 0.16259, 'confiden...","[{'start': 0.76593, 'duration': 0.30136, 'conf...",4.0.0,Linux,...,1,0.543,eJxdm9mB6zoMQ1txCdop9d_YHEBelPl4940Ux6YpEgSX5J...,3.15,eJzNnQvSa7mNpLd0-CaXw-f-lzBfQmXLfaNFhUdTUdMOo8...,4.12,eJxVWQmS3CAM_IqfYCQu__9joQ8xm8rWZMZg0NGSGhHxjv...,1.0,eJxtXA2S5LwKu0qOEP9h-_4XWyMJnJ7aqm_fzHR3EtuAEI...,1.0
2,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,1,"[{'start': 1.26362, 'duration': 1.40759, 'conf...","[{'start': 0.20422, 'duration': 0.36619, 'conf...","[{'start': 0.0, 'duration': 11.98776, 'confide...","[{'start': 0.0, 'duration': 0.08857, 'confiden...","[{'start': 0.20422, 'duration': 0.1831, 'confi...",4.0.0,Linux,...,1,0.152,eJxdmgl2ZLkNBK9SR-C-3P9ijkhWSxr3s-eJLH4uIJBIAJ...,3.15,eJzNvQmWLLmNtLsl50wuh-P-l_A-Q5QqpNvPmaf_PHXUPU...,4.12,eJxVWAlyxDAI-4qfYMDn_z9WkOS0ne5mG8cHhxAQ9z57i2...,1.0,eJyVXAmS5Dquu4qPYO3S_S82BEBKclbPxP8Rr7u6MmUtFA...,1.0
3,4jPy3l0RUwlUI9T5XHBW2m,Mood (feat. iann dior),1,"[{'start': 0.24518, 'duration': 2.61968, 'conf...","[{'start': 0.24518, 'duration': 0.65366, 'conf...","[{'start': 0.0, 'duration': 8.14435, 'confiden...","[{'start': 0.0, 'duration': 0.22045, 'confiden...","[{'start': 0.24518, 'duration': 0.32683, 'conf...",4.0.0,Linux,...,0,0.672,eJxVmgmy5TYOBK-iI3Bf7n-xySy-7vY4wvYnJZHEViiAr-...,3.15,eJzdnQm2HLmSXLcUmIHlYNz_EnTNsz6zyNOJJ-o1RbX6n_...,4.12,eJxdWQmS5EgI-4qfYEjy-v_HFkng6tmYimqX8-IUIsf9ne...,1.0,eJxlWolxJLsOS6VDkKg7_8Q-cUiefb9qXV7PdOsgQRCkVG...,1.0
4,2tGvwE8GcFKwNdAXMnlbfl,happier,1,"[{'start': 1.90661, 'duration': 1.07988, 'conf...","[{'start': 1.17935, 'duration': 0.3577, 'confi...","[{'start': 0.0, 'duration': 4.06404, 'confiden...","[{'start': 0.0, 'duration': 0.93401, 'confiden...","[{'start': 1.17935, 'duration': 0.17885, 'conf...",4.0.0,Linux,...,1,0.648,eJxVmlmi5DYOBK-iI3Bf7n-xiUiWnz3-aDdZEkUCyEQC7F...,3.15,eJzNnWmObTluhLekeViOxv0vwV8oG37dBR_mjwfDBmxWV-...,4.12,eJydWAeS4zgM_AqfIASm_39s0Q1Qksc7W1dX4-LQIIicaN...,1.0,eJxNmwt2ZDkIQ7fyluD_Z_8ba3SFK31OZ5JUqvwwBiGEp9...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173,37dYAkMa4lzRCH6kDbMT1L,We No Speak Americano (Edit),12,"[{'start': 0.46828, 'duration': 1.95247, 'conf...","[{'start': 0.46828, 'duration': 0.48999, 'conf...","[{'start': 0.0, 'duration': 17.74446, 'confide...","[{'start': 0.0, 'duration': 0.20304, 'confiden...","[{'start': 0.46828, 'duration': 0.24499, 'conf...",4.0.0,Linux,...,1,0.151,eJxVmtdhwzAMRFfRCOxl_8XyHijbzFdCWmJBORwAtT5naq...,3.15,eJztvQm2IzmSbLklwwwsB-P-l9BXlJFBLz9J8Md_6VXVpz...,4.12,eJxVWYm15DAIayUlGPDZf2NrJOH5--Yfk8SxMYcQ2L2N9s...,1.0,eJyVWwl2pTgMvMo_At7t-19sVFXyAp8kPe91ZggYIUtl7Q...,1.0
1174,3dxDj8pDPlIHCIrUPXuCeG,Sandstorm,12,"[{'start': 1.5228, 'duration': 1.79451, 'confi...","[{'start': 0.63643, 'duration': 0.44064, 'conf...","[{'start': 0.0, 'duration': 11.69417, 'confide...","[{'start': 0.0, 'duration': 0.38317, 'confiden...","[{'start': 0.63643, 'duration': 

## Unnest columns from audio analysis dataset
The columns are `bars`, `beats`, `sections`, `segments` and `tatums`.

In [3]:
df_bars = df_audio_analysis[['track.id', 'bars']].copy()
df_bars['bars_json'] = df_bars['bars'].apply(lambda x: json.loads(x.replace("\'", "\"")))
df_bars = df_bars.explode('bars_json')

start = df_bars['bars_json'].apply(lambda x: x.get('start'))
duration = df_bars['bars_json'].apply(lambda x: x.get('duration'))
confidence = df_bars['bars_json'].apply(lambda x: x.get('confidence'))
df_bars = df_bars.assign(start = start.values, duration = duration.values, confidence = confidence.values)

df_bars = df_bars[['track.id', 'start', 'duration', 'confidence']].reset_index(drop = True)
df_bars.head()

,track.id,start,duration,confidence
0,4CPYZtb4tX2V03jcsJAZCD,0.68467,2.18383,0.457
1,4CPYZtb4tX2V03jcsJAZCD,2.86850,2.18030,0.462
2,4CPYZtb4tX2V03jcsJAZCD,5.04880,2.18139,0.425
3,4CPYZtb4tX2V03jcsJAZCD,7.23019,2.17856,0.143
4,4CPYZtb4tX2V03jcsJAZCD,9.40875,2.18250,0.455


In [4]:
# # Save tabulated bar data to a file
# df_bars.to_csv('./data/audio_analysis_bar.csv', index = False)

In [5]:
df_beats = df_audio_analysis[['track.id', 'beats']].copy()
df_beats['beats_json'] = df_beats['beats'].apply(lambda x: json.loads(x.replace("\'", "\"")))
df_beats = df_beats.explode('beats_json')

start = df_beats['beats_json'].apply(lambda x: x.get('start'))
duration = df_beats['beats_json'].apply(lambda x: x.get('duration'))
confidence = df_beats['beats_json'].apply(lambda x: x.get('confidence'))
df_beats = df_beats.assign(start = start.values, duration = duration.values, confidence = confidence.values)

df_beats = df_beats[['track.id', 'start', 'duration', 'confidence']].reset_index(drop = True)
df_beats.head()

,track.id,start,duration,confidence
0,4CPYZtb4tX2V03jcsJAZCD,0.13989,0.54478,1.000
1,4CPYZtb4tX2V03jcsJAZCD,0.68467,0.54501,0.895
2,4CPYZtb4tX2V03jcsJAZCD,1.22968,0.54650,0.920
3,4CPYZtb4tX2V03jcsJAZCD,1.77618,0.54592,0.994
4,4CPYZtb4tX2V03jcsJAZCD,2.32210,0.54640,0.984


In [6]:
# # Save tabulated beats data to a file
# df_beats.to_csv('./data/audio_analysis_beats.csv', index = False)

In [7]:
df_sections = df_audio_analysis[['track.id', 'sections']].copy()
df_sections['sections_json'] = df_sections['sections'].apply(lambda x: json.loads(x.replace("\'", "\"")))
df_sections = df_sections.explode('sections_json')

start = df_sections['sections_json'].apply(lambda x: x.get('start'))
duration = df_sections['sections_json'].apply(lambda x: x.get('duration'))
confidence = df_sections['sections_json'].apply(lambda x: x.get('confidence'))
df_sections = df_sections.assign(start = start.values, duration = duration.values, confidence = confidence.values)

df_sections = df_sections[['track.id', 'start', 'duration', 'confidence']].reset_index(drop = True)
df_sections.head()

,track.id,start,duration,confidence
0,4CPYZtb4tX2V03jcsJAZCD,0.00000,18.13542,1.000
1,4CPYZtb4tX2V03jcsJAZCD,18.13542,16.36739,0.744
2,4CPYZtb4tX2V03jcsJAZCD,34.50281,20.17776,0.530
3,4CPYZtb4tX2V03jcsJAZCD,54.68057,16.36896,0.514
4,4CPYZtb4tX2V03jcsJAZCD,71.04952,17.45000,0.090


In [8]:
# # Save tabulated sections data to a file
# df_sections.to_csv('./data/audio_analysis_sections.csv', index = False)

In [9]:
df_segments = df_audio_analysis[['track.id', 'segments']].copy()
df_segments['segments_json'] = df_segments['segments'].apply(lambda x: json.loads(x.replace("\'", "\"")))
df_segments = df_segments.explode('segments_json')

start = df_segments['segments_json'].apply(lambda x: x.get('start'))
duration = df_segments['segments_json'].apply(lambda x: x.get('duration'))
confidence = df_segments['segments_json'].apply(lambda x: x.get('confidence'))
df_segments = df_segments.assign(start = start.values, duration = duration.values, confidence = confidence.values)

df_segments = df_segments[['track.id', 'start', 'duration', 'confidence']].reset_index(drop = True)
df_segments.head()

,track.id,start,duration,confidence
0,4CPYZtb4tX2V03jcsJAZCD,0.00000,0.13338,0.000
1,4CPYZtb4tX2V03jcsJAZCD,0.13338,0.17637,1.000
2,4CPYZtb4tX2V03jcsJAZCD,0.30975,0.20100,0.837
3,4CPYZtb4tX2V03jcsJAZCD,0.51075,0.17324,0.699
4,4CPYZtb4tX2V03jcsJAZCD,0.68399,0.24902,0.620


In [10]:
# # Save tabulated segments data to a file
# df_segments.to_csv('./data/audio_analysis_segments.csv', index = False)

In [11]:
df_tatums = df_audio_analysis[['track.id', 'tatums']].copy()
df_tatums['tatums_json'] = df_tatums['tatums'].apply(lambda x: json.loads(x.replace("\'", "\"")))
df_tatums = df_tatums.explode('tatums_json')

start = df_tatums['tatums_json'].apply(lambda x: x.get('start'))
duration = df_tatums['tatums_json'].apply(lambda x: x.get('duration'))
confidence = df_tatums['tatums_json'].apply(lambda x: x.get('confidence'))
df_tatums = df_tatums.assign(start = start.values, duration = duration.values, confidence = confidence.values)

df_tatums = df_tatums[['track.id', 'start', 'duration', 'confidence']].reset_index(drop = True)
df_tatums.head()

,track.id,start,duration,confidence
0,4CPYZtb4tX2V03jcsJAZCD,0.13989,0.27239,1.000
1,4CPYZtb4tX2V03jcsJAZCD,0.41228,0.27239,1.000
2,4CPYZtb4tX2V03jcsJAZCD,0.68467,0.27250,0.895
3,4CPYZtb4tX2V03jcsJAZCD,0.95718,0.27250,0.895
4,4CPYZtb4tX2V03jcsJAZCD,1.22968,0.27325,0.920


In [12]:
# # Save tabulated tatums data to a file
# df_tatums.to_csv('./data/audio_analysis_tatums.csv', index = False)